<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Events_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
res = requests.get('https://www.visitberlin.de/en/event-calendar-berlin?page=0')
soup = BeautifulSoup(res.text, 'html.parser')
soup

In [ ]:
base_url = "https://www.visitberlin.de"
# Extract event name
event_name = soup.find('a', class_='teaser-search__mainlink').get('title', '').strip()
picture_link = base_url + soup.find('img', class_='teaser-search__img fluid-img').get('src')
category = soup.find('div', class_='teaser-search__category').get_text(strip=True)
address = soup.find('span', class_='teaser-search__print-address teaser-search__print-info').get_text(strip=True)
website = soup.find('p', class_='teaser-search__print-link teaser-search__print-info').get_text(strip=True)
time = None
time_tag = soup.find('p', class_='teaser-search__time me')
if time_tag:
    # Extract the time if the tag exists
    time = time_tag.select_one('span.me__content').get_text(strip=True)

start_date, end_date = None, None
date_tag = soup.find('p', class_='teaser-search__date')
if date_tag:
  start_date = date_tag.find('time').get_text(strip=True)
  end_date = date_tag.find_all('time')[1].get_text(strip=True)
# Extract picture URL
# picture_tag = soup.find('picture')
# if picture_tag:
#     picture_url = picture_tag.find('source', media="all and (min-width: 1560px)")['srcset'].split()[0]
# else:
#     picture_url = None

In [ ]:
picture_link

'https://www.visitberlin.de/system/files/styles/visitberlin_teaser_search_visitberlin_mobile_1x/private/event_images/vb-60-ff5d39a8-8617-4fd4-a53c-a2a1b7e80cbf.jpg?h=43ec1315&itok=PACrzZn_'

In [12]:
hamburgUrl = "https://www.hamburg-travel.com/see-explore/events/events-calendar/js.api?filter[date]=02.10.2024&filter[district]=all&page=0&filter[distance]=50"
res = requests.get(hamburgUrl)
soup = BeautifulSoup(res.text, 'html.parser')
hamburgEvents = soup.find_all('article', class_='listTeaser-event')
len(hamburgEvents)

3

In [17]:
muenchenUrl = "https://www.muenchen.de/en/veranstaltungen/event?page=0"
res = requests.get(muenchenUrl)
soup = BeautifulSoup(res.text, 'html.parser')
muenchenEvents = soup.find_all('li', class_='m-listing__list-item')
len(muenchenEvents)

0

In [15]:
4391/30

146.36666666666667